In [232]:
import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import itertools as it
%matplotlib inline

In [137]:
def routes_jalan(G, routes, iterasi):
    if iterasi == 0:
        t = 'travel_time_awal'
    else:
        t = 'travel_time' + str(iterasi)
    list_jalan = []
    for i in range(len(routes)-1):
        minimum = G[routes[i]][routes[i+1]][0][t]
        j = 0
        keys = 0
        for j in range(len(G.get_edge_data(routes[i], routes[i+1]))):
            if G.get_edge_data(routes[i], routes[i+1], j)[t] < minimum:
                minimum = G.get_edge_data(routes[i], routes[i+1], j)[t]
                keys = j
        list_jalan.append(keys)
    
    return list_jalan

In [185]:
def updating_mainflow(G, routes, iterasi):

    nx.set_edge_attributes(G, 0, 'flow1')

    rute_jalan_key = routes_jalan(G, routes, iterasi)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"flow1": flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [210]:
def updating_auxflow(G, routes,iterasi):

    nx.set_edge_attributes(G, 0, 'auxflow'+str(iterasi))

    rute_jalan_key = routes_jalan(G, routes, iterasi)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"auxflow"+str(iterasi): flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [49]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr_idn(row, iterasi):
    if np.isnan(row['flow' + str(iterasi)]):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+0.15*(row['flow'+str(iterasi)]/row['capacity'])**4)
        except ValueError:
            return row['travel_time']

In [6]:
def line_search(row, iterasi, a, alpha, beta):
    t=row['travel_time_awal']
    x=row['flow'+str(iterasi)]
    y=row['auxflow'+str(iterasi)]
    c=row['capacity']
    return -t*(x-y)*(alpha*(((x+a*(y-x))/c)**beta)+1)

In [7]:
def update(row, alpha, iterasi):
    return row['flow'+str(iterasi)] + alpha * (row['auxflow'+str(iterasi)] - row['flow'+str(iterasi)])

In [54]:
def bisection(G, iterasi, xl=0, xr=1, delta=0.0002, alpha=0.15, beta=4):
    n = 0
    df = nx.to_pandas_edgelist(G)
    condition = True
    while condition:
        n += 1
        x = (xl+xr)/2
        if df.apply(lambda row: line_search(row, iterasi, x, alpha=alpha, beta=beta), axis = 1).sum() <= 0:
            xl = x
        else:
            xr = x
        condition = abs(xr-xl) > 2*delta
        # print('|xl-xr| =', abs(xr-xl),' xl =', xl, ' xr=',xr)
    # print(f"number of iteration in bisection method is {n} with alpha = {(xr+xl)/2}")
    return (xr+xl)/2

# Convev Combinations Algorithm

In [213]:
def CCA(G, source, target, convergence):

    # iterasi 0
    routes = nx.shortest_path(G, source, target, weight='travel_time_awal')
    G = updating_mainflow(G, routes, 0)

    condition = True
    iterasi = 1

    while condition == True:
        unique_edge = set(G.edges())

        for i in unique_edge:
            for j in range(len(G.get_edge_data(i[0],i[1]))):
                attrib = {(i[0], i[1], j): {"travel_time"+str(iterasi): lpr_idn(G[i[0]][i[1]][j], iterasi)}}
                nx.set_edge_attributes(G, attrib)

        # calculate auxiliary flow
        routes = nx.shortest_path(G, source, target, weight='travel_time'+str(iterasi))
        G = updating_auxflow(G, routes, iterasi)

        # updating the flow
        alpha = bisection(G, iterasi)
        if alpha < convergence:
            condition = False

        print('iterasi', iterasi, ' alpha =', alpha)
        df = nx.to_pandas_edgelist(G)
        flow_baru = df.apply(lambda row: update(row, alpha, iterasi), axis=1)
        new_flow = 'flow' + str(iterasi+1)

        index = 0
        for i in unique_edge:
            for j in range(len(G.get_edge_data(i[0],i[1]))):
                    attrib = {(i[0], i[1], j): {new_flow: flow_baru[index]}}
                    nx.set_edge_attributes(G, attrib)
                    index += 1
        iterasi += 1
                
    return G

In [236]:
# Contoh
G = nx.MultiDiGraph()
key = G.add_edge(1,2, capacity = 2, travel_time_awal=10)
key2 = G.add_edge(1,2, capacity = 4, travel_time_awal=20)
key3 = G.add_edge(1,2, capacity = 3, travel_time_awal=25)
# key4 = G.add_edge(2,3, capacity = 5, travel_time_awal=37)
# key5 = G.add_edge(2,3, capacity = 5, travel_time_awal=15)
flow = 10

H = CCA(G,1,2, 0.01)
nx.to_pandas_edgelist(H)

iterasi 1  alpha = 0.5965576171875
iterasi 2  alpha = 0.1610107421875
iterasi 3  alpha = 0.0355224609375
iterasi 4  alpha = 0.0203857421875
iterasi 5  alpha = 0.0072021484375


,source,target,travel_time5,travel_time1,flow1,auxflow2,flow2,travel_time_awal,flow4,travel_time3,auxflow4,flow6,flow3,travel_time4,capacity,travel_time2,auxflow1,auxflow3,flow5,auxflow5
0,1,2,24.823201,947.5,10,0,4.034424,10,3.619825,22.306171,0,3.592515,3.384838,26.096135,2,34.836898,0,10,3.546032,10
1,1,2,25.860104,20.0,0,0,5.965576,20,4.827262,27.353879,0,4.694797,5.005054,26.363337,4,34.841947,10,0,4.728855,0
2,1,2,25.410030,25.0,0,10,0.000000,25,1.552912,25.311147,10,1.712688,1.610107,25.269237,3,25.000000,0,0,1.725113,0
